In [147]:
i:read0`:test1.txt
i

"seeds: 79 14 55 13"
""
"seed-to-soil map:"
"50 98 2"
"52 50 48"
""
"soil-to-fertilizer map:"
"0 15 37"
"37 52 2"
"39 0 15"
""
"fertilizer-to-water map:"
"49 53 8"
"0 11 42"
"42 0 7"
"57 7 4"
""
"water-to-light map:"
"88 18 7"
"18 25 70"
""
"light-to-temperature map:"
..


In [148]:
/ maps are sequential - so just apply in order
i where i like "*-to-*"

"seed-to-soil map:"
"soil-to-fertilizer map:"
"fertilizer-to-water map:"
"water-to-light map:"
"light-to-temperature map:"
"temperature-to-humidity map:"
"humidity-to-location map:"


In [149]:
/ parse first line
seeds:get last ":" vs i 0
seeds

/ remove seeds line
rs:2_i

79 14 55 13


In [150]:
/ cut by empty lines
elc:(0,where rs like "") cut rs
elc

("seed-to-soil map:";"50 98 2";"52 50 48")
("";"soil-to-fertilizer map:";"0 15 37";"37 52 2";"39 0 15")
("";"fertilizer-to-water map:";"49 53 8";"0 11 42";"42 0 7";"57 7 4")
("";"water-to-light map:";"88 18 7";"18 25 70")
("";"light-to-temperature map:";"45 77 23";"81 45 19";"68 64 13")
("";"temperature-to-humidity map:";"0 69 1";"1 0 69")
("";"humidity-to-location map:";"60 56 37";"56 93 4")


In [151]:
/ remove empty lines and headers
elh:_'[1,1_(count elc)#2;elc]
elh

("50 98 2";"52 50 48")
("0 15 37";"37 52 2";"39 0 15")
("49 53 8";"0 11 42";"42 0 7";"57 7 4")
("88 18 7";"18 25 70")
("45 77 23";"81 45 19";"68 64 13")
("0 69 1";"1 0 69")
("60 56 37";"56 93 4")


In [152]:
/ parse numbers
pn:get''[elh]
pn

(50 98 2;52 50 48)
(0 15 37;37 52 2;39 0 15)
(49 53 8;0 11 42;42 0 7;57 7 4)
(88 18 7;18 25 70)
(45 77 23;81 45 19;68 64 13)
(0 69 1;1 0 69)
(60 56 37;56 93 4)


In [153]:
/ build range dict
{(y+til z)!x+til z} . first first pn

98| 50
99| 51


In [154]:
/ joined dict for the whole map
{x,{(y+til z)!x+til z} . y}/[()!();first pn]

98| 50
99| 51
50| 52
51| 53
52| 54
53| 55
54| 56
55| 57
56| 58
57| 59
58| 60
59| 61
60| 62
61| 63
62| 64
63| 65
64| 66
65| 67
66| 68
67| 69
68| 70
69| 71
..


In [155]:
/ get all maps
maps:{x,{(y+til z)!x+til z} . y}/[()!();] each pn
maps

98 99 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73..
15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40..
53 54 55 56 57 58 59 60 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28..
18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43..
77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 45 46 47..
69 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 ..
56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81..


In [156]:
/ support default mapping
map:{?[not x in key y;x;y x]}
map[10 20 50;maps 0]

10 20 52


In [157]:
/ part 1
min map/[seeds;maps]

35


In [158]:
/ build seeds for 2
seeds2:raze {x+til y} .' 2 cut seeds
seeds2

79 80 81 82 83 84 85 86 87 88 89 90 91 92 55 56 57 58 59 60 61 62 63 64 65 66..


In [159]:
/ part 1
min map/[seeds2;maps]

46


In [160]:
/ part 2 was too big, need to map intervals
/ first check if intervals are disjoint, adjacent, etc.
asc {(x 1;-1+sum x 1 2;.[-]x 0 1)} each 0N!first pn
/ corresponds to intervals [0;49],[50;97];[98;99];[100;) -> 0, 2, -48, 0

(50 98 2;52 50 48)


50 97 2  
98 99 -48


In [161]:
seedInts:asc {(x 0;-1+sum x 0 1)} each 2 cut seeds
seedInts

55 67
79 92


In [162]:
/ use step-dictionary to translate intervals
d:`s#(asc key d)#d:{x[y 1]:y[0]-y 1;end:sum y 1 2;if[not end in key x;x[end]:0];x}/[enlist[0]!enlist 0;first pn]
d

0  | 0
50 | 2
98 | -48
100| 0


In [163]:
map:{[m;l;r]
    li:key[m] binr l;
    ri:key[m] bin r;
    if[li>ri;
        :enlist (l;r)+m l
     ];
    newBounds:asc l,(asc raze -1 0+/:key[m] li+til 1+ri-li),r;
    asc 2 cut newBounds+m newBounds
 }
map[d]./:(45 49;45 55;45 105)

,45 49
`s#(45 49;52 57)
`s#(45 49;50 51;52 99;100 105)


In [164]:
pm:{x[y 1]:y[0]-y 1;end:sum y 1 2;if[not end in key x;x[end]:0];x}
maps:{`s#(asc key d)#d:pm/[enlist[0]!enlist 0;x]} each pn
maps

`s#`s#0 50 98 100!0 2 -48 0
`s#`s#0 15 52 54!39 -15 -15 0
`s#`s#0 7 11 53 61!42 50 -11 -4 0
`s#`s#0 18 25 95!0 70 -7 0
`s#`s#0 45 64 77 100!0 36 4 -32 0
`s#`s#0 69 70!1 -69 0
`s#`s#0 56 93 97!0 4 -37 0


In [165]:
min raze {raze map[y]./:x}/[seedInts;maps]

46


In [166]:
/ part 2 combined
i:read0`:input.txt
seeds:get last ":" vs i 0
pn:get''[_'[1,1_(count elc)#2;elc:(0,where rs like "") cut rs:2_i]]

seedInts:asc {(x 0;-1+sum x 0 1)} each 2 cut seeds
maps:{`s#(asc key d)#d:pm/[enlist[0]!enlist 0;x]} each pn
min raze {raze map[y]./:x}/[seedInts;maps]

78775051
